In [1]:
from xml.dom import minidom
import editdistance
import random

# an action here is a gui element
def get_action_vector(arg_action):
    str_index = "{0:08b}".format(int(arg_action.index))
    vec_index = [1 if p == "1" else 0 for p in str_index]
    vec_checkable = [1 if arg_action.checkable == "true" else 0]
    vec_checked = [1 if arg_action.checked == "true" else 0]
    vec_clickable = [1 if arg_action.clickable == "true" else 0]
    vec_enabled = [1 if arg_action.enabled == "true" else 0]
    vec_focusable = [1 if arg_action.focusable == "true" else 0]
    vec_focused = [1 if arg_action.focused == "true" else 0]
    vec_scrollable = [1 if arg_action.scrollable == "true" else 0]
    vec_long_clickable = [1 if arg_action.long_clickable == "true" else 0]
    vec_password = [1 if arg_action.password == "true" else 0]
    vec_selected = [1 if arg_action.selected == "true" else 0]
    vec_visible_to_user = [1 if arg_action.visible_to_user == "true" else 0]
    return vec_index + vec_checkable + vec_checked + vec_clickable + vec_enabled + \
           vec_focusable + vec_focused + vec_scrollable + vec_long_clickable + \
           vec_password + vec_selected + vec_visible_to_user

def get_action_pool_matrix(arg_action_pool):
    action_pool_matrix = []
    for i in range(len(arg_action_pool)):
        action_pool_matrix.append( get_action_vector(arg_action_pool[i]) )
    return action_pool_matrix

# a state here is a windows hierarchy string
def get_state_matrix(arg_state, arg_maxn_row):
    state_nodes = minidom.parseString(arg_state).getElementsByTagName('node')
    state_matrix = []
    for i in range(len(state_nodes)):
        node_i = state_nodes[i]
        str_index = "{0:08b}".format(int(node_i.attributes['index'].value))
        vec_index = [1 if p == "1" else 0 for p in str_index]
        vec_checkable = [1 if node_i.attributes["checkable"].value == "true" else 0]
        vec_checked = [1 if node_i.attributes["checked"].value == "true" else 0]
        vec_clickable = [1 if node_i.attributes["clickable"].value == "true" else 0]
        vec_enabled = [1 if node_i.attributes["enabled"].value == "true" else 0]
        vec_focusable = [1 if node_i.attributes["focusable"].value == "true" else 0]
        vec_focused = [1 if node_i.attributes["focused"].value == "true" else 0]
        vec_scrollable = [1 if node_i.attributes["scrollable"].value == "true" else 0]
        vec_long_clickable = [1 if node_i.attributes["long-clickable"].value == "true" else 0]
        vec_password = [1 if node_i.attributes["password"].value == "true" else 0]
        vec_selected = [1 if node_i.attributes["selected"].value == "true" else 0]
        vec_visible_to_user = [1 if node_i.attributes["visible-to-user"].value == "true" else 0]
        state_matrix.append( vec_index + vec_checkable + vec_checked + vec_clickable + vec_enabled + \
                             vec_focusable + vec_focused + vec_scrollable + vec_long_clickable + \
                             vec_password + vec_selected + vec_visible_to_user )
    assert len(state_matrix)<=arg_maxn_row
    for i in range(arg_maxn_row-len(state_matrix)):
        state_matrix.append([0 for _ in range(19)])
    return state_matrix

def get_reward0(arg_state0, arg_state1):
    # need to remove system ui, such as clock, since it varies every minute/second
    def remove_system_ui(args_dom):
        for node in args_dom:
            if node.hasAttribute('package') and node.getAttribute('package') == "com.android.systemui":
                parent = node.parentNode
                parent.removeChild(node)
    # states should be xml strings
    dom0 = minidom.parseString(arg_state0)
    dom1 = minidom.parseString(arg_state1)
    pre_nodes0 = dom0.getElementsByTagName('node')
    pre_nodes1 = dom1.getElementsByTagName('node')
    remove_system_ui(pre_nodes0)
    remove_system_ui(pre_nodes1)
    nodes0 = dom0.getElementsByTagName('node')
    nodes1 = dom1.getElementsByTagName('node')
    xmls0 = [p.toxml() for p in nodes0]
    xmls1 = [p.toxml() for p in nodes1]
    # print("# xmls0: {}, xmls1: {}, edit: {}".format(len(xmls0), len(xmls1), editdistance.eval(xmls0, xmls1)))
    return 1.0 - float( editdistance.eval(xmls0, xmls1) ) / float( max(len(xmls0), len(xmls1)) )

In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device: {}".format(device))
print("torch version: {}".format(torch.__version__))

device: cpu
torch version: 1.6.0


In [3]:
class NeuralAgent(nn.Module):
    def __init__(self, spec_dims=None, maxn_nodes=None):
        super(NeuralAgent, self).__init__()
        # (kernel size, number of kernels)
        self.n_kernels = [10, 10, 10]
        self.kernel_sizes = [1, 2, 3]
        self.convs = nn.ModuleList([
            nn.Conv1d(spec_dims, self.n_kernels[i], self.kernel_sizes[i], padding=0)
            for i in range(len(self.kernel_sizes))
        ])
        self.maxlen = maxn_nodes
        self.pools = nn.ModuleList([
            nn.MaxPool1d(self.maxlen+1-self.kernel_sizes[i], padding=0)
            for i in range(len(self.kernel_sizes))
        ])
        self.linear = nn.Linear(sum(self.n_kernels), spec_dims)
        
    def forward(self, node_seq):
        # node_seq: (B, n_nodes, spec_dims), usually B=1
        B = node_seq.shape[0] # batch size
        tmp0 = node_seq.transpose(1,2) # (N,C,L), which is (B, spec_dims, n_nodes)
        # print("# tmp0: {}".format(tmp0.shape))
        # (B, n_kernels, maxlen-i)
        tmp1s = [
            F.relu(self.convs[i](tmp0))
            for i in range(len(self.convs))
        ]
        # print("# tmp1s: {}".format(tmp1s[0].shape))
        # (B, n_kernels, 1)
        tmp2s = [
            self.pools[i](tmp1s[i])
            for i in range(len(tmp1s))
        ]
        # (B, n_kernels)
        tmp3s = [
            tmp2s[i].view(B, self.n_kernels[i])
            for i in range(len(tmp2s))
        ]
        tmp4 = torch.cat(tmp3s, 1) # (B, sum(n_kernels))
        tmp5 = torch.sigmoid(self.linear(tmp4)) # (B, spec_dims)
        return tmp5

In [10]:
# 1. roll out an action sequence
# 2. compute reward
# 3. policy gradient back propagation
def rollout(env, agent, optimizer, maxn_steps, maxn_nodes, target_state):
    # note: remember to clear the state
    # fixme: it seems not working right now
    env.launch_app()
    
    rollout_outputs = []
    rollout_actions = []
    rollout_action_ids = []
    
    for i in range(maxn_steps):
        time.sleep(1)
        state_i = env.get_current_state()
        if get_reward0(state_i, target_state) == 1:
            print("# reach target state after {} actions".format(len(rollout_outputs)))
            # already reach the target state
            break
            
        pool_i = env.get_available_actionable_elements(state_i)
        n_actions = len(pool_i)
        if n_actions == 0:
            print("# no action is found, terminate.")
            # no available actions any more
            break
        
        # (B=1, n_nodes, spec_dims), wrap a [] to make B=1
        m_state = np.asarray( [ get_state_matrix(state_i, maxn_nodes) ] )
        
        # (n_actions, spec_dims)
        m_pool = np.asarray( get_action_pool_matrix(pool_i) ) 
        
        t_state = Variable(torch.tensor(m_state, dtype=torch.float).to(device))
        t_pool = Variable(torch.tensor(m_pool, dtype=torch.float).to(device))
        
        # print(t_state.shape)
        # print(t_pool.shape)
        
        agent.train()
        # (B=1, spec_dims)
        t_output = agent(t_state)
        
        # ====> using cosine similarity
        # (n_actions, spec_dims)
        # t0_output = t_output.expand_as(t_pool)
        # t_cos = F.cosine_similarity(t0_output, t_pool, dim=1)
        # t_act = F.log_softmax(t_cos, dim=0)
        # ====> directly mm similarity
        t0_output = t_output.view(-1,1)
        t_mm = torch.mm(t_pool, t0_output)
        t_act = F.log_softmax(t_mm, dim=0)
        # print("# t_act: {}".format(t_act))
        
        if random.random()>0.8:
            # explore
            selected_action_id = random.choice(list(range(len(pool_i))))
            print("# selected_action_id (rnd): {}, log-sim: {}".format(selected_action_id, t_act[selected_action_id]))
        else:
            # exploit
            t_probs = t_act.exp().flatten().tolist()
            selected_action_id = random.choices(list(range(len(pool_i))), weights=t_probs, k=1)[0]
            # selected_action_id = torch.argmax(t_act, dim=0).tolist()
            print("# selected_action_id (mul): {}, log-sim: {}".format(selected_action_id, t_act[selected_action_id]))
        
        
        
        # perform action
        env.perform_action(pool_i[selected_action_id])
        next_state = env.get_current_state()
        
        # store the choices
        rollout_outputs.append(t_act)
        rollout_actions.append(pool_i)
        rollout_action_ids.append(selected_action_id)
        
    # here we use the final reward as the cumulative reward
    # which is for the present a brute-force version of reward
    final_state = env.get_current_state()
    # fixme: may need to award shorter paths
    # final_reward = ( get_reward0(final_state, target_state) -0.5 ) *2 # scale to [-1,1]
    final_reward = ( get_reward0(final_state, target_state) -0.5 ) *2 / len(rollout_outputs) # scale to [-1,1]
    print("# final reward: {}".format(final_reward))
    
    rollout_loss = []
    current_reward = final_reward
    # reverse from the last to first
    for i in range(len(rollout_outputs))[::-1]:
        rollout_loss.append( current_reward * (-rollout_outputs[i][rollout_action_ids[i]]) )
        current_reward *= 0.8 # decay
    rollout_loss = rollout_loss[::-1]
    
    optimizer.zero_grad()
    # loss = sum(rollout_loss) / len(rollout_loss)
    loss = sum(rollout_loss)
    loss.backward()
    optimizer.step()
        
        

In [10]:
from main import *

MarthaEnv = os.path.dirname(os.getcwd())
OUTPUT_DIR = os.path.join(MarthaEnv, "results")

args = {
    "path": "../test/com.github.cetoolbox_11/com.github.cetoolbox_11_instrumented.signed.apk",
    "output": "../results/",
}

if args["path"] is not None:
    pyaxmlparser_apk = APK(args["path"])
    apk_base_name = os.path.splitext(os.path.basename(args["path"]))[0]

else:
    parser.print_usage()

if args["output"] is not None:
    OUTPUT_DIR = args["output"]

output_dir = os.path.join(OUTPUT_DIR, apk_base_name)

if os.path.exists(output_dir):
    rmtree(output_dir)

if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# Setting the path for log file
log_path = os.path.join(output_dir, 'analysis.log')
log = init_logging('analyzer.%s' % apk_base_name, log_path, file_mode='w', console=True)

# Record analysis start time
now = datetime.datetime.now()
analysis_start_time = now.strftime(DATE_FORMAT)
info('Analysis started at: %s' % analysis_start_time)
start_time = time.time()

# Get the serial for the device attached to ADB
device_serial = get_device_serial(log)

if device_serial is None:
    log.warning("Device is not connected!")
    sys.exit(1)

# Initialize the uiautomator device object using the device serial
uiautomator_device = u2.connect(device_serial)
run_adb_as_root(log)
apk_obj = Apk(args["path"], uiautomator_device, log)
apk_obj.launch_app()

[INFO] | 2020-12-07 09:52:16 AM | analyzer.com.github.cetoolbox_11_instrumented.signed | Adb is running with root priviledges now!


[#] Analysis started at: 2020-12-07 09:52:16 AM


[INFO] | 2020-12-07 09:52:17 AM | analyzer.com.github.cetoolbox_11_instrumented.signed | APK installtion done for com.github.cetoolbox_11_instrumented.signed.apk
[INFO] | 2020-12-07 09:52:17 AM | analyzer.com.github.cetoolbox_11_instrumented.signed | Kill the current app if already spawned!
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/joseph/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/joseph/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/joseph/Desktop/UCSB/20summer/MarthaEnv/uiautomator_helper/apk.py", line 97, in start_logging
    if "TRAIN DATA" in line or "TEST DATA" in line:
TypeError: a bytes-like object is required, not 'str'

[INFO] | 2020-12-07 09:52:18 AM | analyzer.com.github.cetoolbox_11_instrumented.signed | APK is already killed
[INFO] | 2020-12-07 09:52:18 AM | analyzer.com.github.cetoolbox_11_instrumented.signed 

In [19]:
wh = apk_obj.get_current_state()
aas = apk_obj.get_available_actionable_elements(wh)

In [20]:
aas

In [28]:
apk_obj.perform_action(aas[-2])

In [24]:
dir(aas[-1])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'bounds',
 'checkable',
 'checked',
 'class_name',
 'clickable',
 'compute_coordinate',
 'content_desc',
 'enabled',
 'focusable',
 'focused',
 'index',
 'long_clickable',
 'node',
 'package',
 'password',
 'resource_id',
 'scrollable',
 'selected',
 'setup',
 'text',
 'visible_to_user',
 'x',
 'y']

In [34]:
aas[-6].content_desc

''

In [50]:
dir(aas[-6].node)

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'attrib',
 'clear',
 'extend',
 'find',
 'findall',
 'findtext',
 'get',
 'getchildren',
 'getiterator',
 'insert',
 'items',
 'iter',
 'iterfind',
 'itertext',
 'keys',
 'makeelement',
 'remove',
 'set',
 'tag',
 'tail',
 'text']

In [64]:
aas[-5].node

<Element 'node' at 0x7f9b16613ae8>

In [37]:
apk_obj.get_reached_goal_states("train")

[INFO] | 2020-12-07 10:26:25 AM | analyzer.com.github.cetoolbox_11_instrumented.signed | Old logcat messages cleared!


[]

In [38]:
wh

'<?xml version=\'1.0\' encoding=\'UTF-8\' standalone=\'yes\' ?>\r\n<hierarchy rotation="0">\r\n  <node index="0" text="" resource-id="com.android.systemui:id/navigation_bar_frame" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,2040][1080,2160]">\r\n    <node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,2040][1080,2160]">\r\n      <node index="0" text="" resource-id="com.android.systemui:id/navigation_inflater" class="android.widget.FrameLayout" package="com.android.sys

In [ ]:
uiautomator_device = u2.connect(device_serial)
run_adb_as_root(log)
apk_obj = Apk(APK_PATH, uiautomator_device, log)
apk_obj.launch_app()

In [8]:
debug_final_state = """<?xml version=\'1.0\' encoding=\'UTF-8\' standalone=\'yes\' ?>\r\n<hierarchy rotation="0">\r\n  <node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="com.example.validrec" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,0][1080,2040]">\r\n    <node index="0" text="" resource-id="" class="android.widget.LinearLayout" package="com.example.validrec" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,0][1080,2040]">\r\n      <node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="com.example.validrec" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,60][1080,2040]">\r\n        <node index="0" text="" resource-id="com.example.validrec:id/action_bar_root" class="android.widget.LinearLayout" package="com.example.validrec" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,60][1080,2040]">\r\n          <node index="0" text="" resource-id="android:id/content" class="android.widget.FrameLayout" package="com.example.validrec" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,60][1080,2040]">\r\n            <node index="0" text="" resource-id="com.example.validrec:id/coordinator" class="android.view.ViewGroup" package="com.example.validrec" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,60][1080,2040]">\r\n              <node index="0" text="enter secret message" resource-id="com.example.validrec:id/textView" class="android.widget.TextView" package="com.example.validrec" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,60][1080,1698]" />\r\n            </node>\r\n          </node>\r\n        </node>\r\n      </node>\r\n    </node>\r\n  </node>\r\n  <node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,0][1080,60]">\r\n    <node index="0" text="" resource-id="com.android.systemui:id/scrim_in_front" class="android.view.View" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="true" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,0][1080,60]" />\r\n    <node index="1" text="" resource-id="com.android.systemui:id/scrim_behind" class="android.view.View" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="true" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,0][1080,60]" />\r\n    <node index="2" text="" resource-id="com.android.systemui:id/status_bar_container" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,0][1080,60]">\r\n      <node index="0" text="" resource-id="com.android.systemui:id/status_bar" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,0][1080,60]">\r\n        <node index="0" text="" resource-id="com.android.systemui:id/status_bar_contents" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[20,0][1060,60]">\r\n          <node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[35,0][540,60]">\r\n            <node index="0" text="" resource-id="com.android.systemui:id/status_bar_left_side" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[35,0][540,60]">\r\n              <node index="0" text="10:55" resource-id="com.android.systemui:id/clock" class="android.widget.TextView" package="com.android.systemui" content-desc="10:55 PM" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[35,0][142,60]" />\r\n              <node index="1" text="" resource-id="com.android.systemui:id/notification_icon_area" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[142,0][540,60]">\r\n                <node index="0" text="" resource-id="com.android.systemui:id/notification_icon_area_inner" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[142,0][540,60]">\r\n                  <node index="0" text="" resource-id="com.android.systemui:id/notificationIcons" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[142,0][540,60]">\r\n                    <node index="0" text="" resource-id="" class="android.widget.ImageView" package="com.android.systemui" content-desc="Android Setup notification: Preparing for setup…" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[142,0][197,60]" />\r\n                    <node index="1" text="" resource-id="" class="android.widget.ImageView" package="com.android.systemui" content-desc="ATX notification: UIAutomator" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[197,0][252,60]" />\r\n                    <node index="2" text="" resource-id="" class="android.widget.ImageView" package="com.android.systemui" content-desc="Settings notification: Virtual SD card" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[252,0][307,60]" />\r\n                  </node>\r\n                </node>\r\n              </node>\r\n            </node>\r\n          </node>\r\n          <node index="1" text="" resource-id="com.android.systemui:id/system_icon_area" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[540,0][1045,60]">\r\n            <node index="0" text="" resource-id="com.android.systemui:id/system_icons" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[540,0][1045,60]">\r\n              <node index="0" text="" resource-id="com.android.systemui:id/statusIcons" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[540,0][1024,60]">\r\n                <node index="0" text="" resource-id="com.android.systemui:id/wifi_combo" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="Wifi signal full." checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[924,2][971,57]">\r\n                  <node index="0" text="" resource-id="com.android.systemui:id/wifi_group" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[930,2][971,57]">\r\n                    <node index="0" text="" resource-id="com.android.systemui:id/wifi_combo" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[930,10][971,48]">\r\n                      <node index="0" text="" resource-id="com.android.systemui:id/wifi_signal" class="android.widget.ImageView" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[930,10][971,48]" />\r\n                    </node>\r\n                  </node>\r\n                </node>\r\n                <node index="1" text="" resource-id="com.android.systemui:id/mobile_combo" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="Phone two bars." checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[971,2][1009,57]">\r\n                  <node index="0" text="" resource-id="com.android.systemui:id/mobile_group" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[971,2][1009,57]">\r\n                    <node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[971,10][1009,48]">\r\n                      <node index="0" text="" resource-id="com.android.systemui:id/mobile_signal" class="android.widget.ImageView" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[971,10][1009,48]" />\r\n                    </node>\r\n                  </node>\r\n                </node>\r\n              </node>\r\n              <node index="1" text="" resource-id="com.android.systemui:id/battery" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="Battery 100 percent." checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[1024,0][1045,60]">\r\n                <node index="0" text="" resource-id="" class="android.widget.ImageView" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[1024,13][1045,46]" />\r\n              </node>\r\n            </node>\r\n          </node>\r\n        </node>\r\n      </node>\r\n    </node>\r\n  </node>\r\n  <node index="0" text="" resource-id="com.android.systemui:id/navigation_bar_frame" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,2040][1080,2160]">\r\n    <node index="0" text="" resource-id="" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,2040][1080,2160]">\r\n      <node index="0" text="" resource-id="com.android.systemui:id/navigation_inflater" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,2040][1080,2160]">\r\n        <node index="0" text="" resource-id="com.android.systemui:id/rot0" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,2040][1080,2160]">\r\n          <node index="0" text="" resource-id="com.android.systemui:id/nav_buttons" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[0,2040][1080,2160]">\r\n            <node index="0" text="" resource-id="com.android.systemui:id/ends_group" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[20,2040][1060,2160]">\r\n              <node index="0" text="" resource-id="" class="android.widget.RelativeLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[40,2040][140,2160]" />\r\n              <node index="1" text="" resource-id="" class="android.widget.RelativeLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[140,2040][340,2160]">\r\n                <node index="0" text="" resource-id="com.android.systemui:id/back" class="android.widget.ImageView" package="com.android.systemui" content-desc="Back" checkable="false" checked="false" clickable="true" enabled="true" focusable="true" focused="false" scrollable="false" long-clickable="true" password="false" selected="false" visible-to-user="true" bounds="[140,2040][340,2160]" />\r\n              </node>\r\n              <node index="2" text="" resource-id="" class="android.widget.RelativeLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[740,2040][940,2160]">\r\n                <node index="0" text="" resource-id="com.android.systemui:id/recent_apps" class="android.widget.ImageView" package="com.android.systemui" content-desc="Overview" checkable="false" checked="false" clickable="true" enabled="true" focusable="true" focused="false" scrollable="false" long-clickable="true" password="false" selected="false" visible-to-user="true" bounds="[740,2040][940,2160]" />\r\n              </node>\r\n              <node index="3" text="" resource-id="" class="android.widget.RelativeLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[940,2040][1040,2160]">\r\n                <node NAF="true" index="0" text="" resource-id="com.android.systemui:id/menu_container" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="true" enabled="true" focusable="true" focused="false" scrollable="false" long-clickable="true" password="false" selected="false" visible-to-user="true" bounds="[940,2040][1040,2160]" />\r\n              </node>\r\n            </node>\r\n            <node index="1" text="" resource-id="com.android.systemui:id/center_group" class="android.widget.LinearLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[420,2040][660,2160]">\r\n              <node index="0" text="" resource-id="com.android.systemui:id/home" class="android.widget.FrameLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="true" enabled="true" focusable="true" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[440,2040][640,2160]">\r\n                <node index="0" text="" resource-id="com.android.systemui:id/home_button" class="android.widget.ImageView" package="com.android.systemui" content-desc="Home" checkable="false" checked="false" clickable="true" enabled="true" focusable="true" focused="false" scrollable="false" long-clickable="true" password="false" selected="false" visible-to-user="true" bounds="[440,2040][640,2160]" />\r\n                <node index="1" text="" resource-id="" class="android.widget.RelativeLayout" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[440,2040][640,2160]" />\r\n                <node index="2" text="" resource-id="com.android.systemui:id/white_cutout" class="android.widget.ImageView" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[517,2077][562,2122]" />\r\n                <node index="3" text="" resource-id="com.android.systemui:id/white" class="android.widget.ImageView" package="com.android.systemui" content-desc="" checkable="false" checked="false" clickable="false" enabled="true" focusable="false" focused="false" scrollable="false" long-clickable="false" password="false" selected="false" visible-to-user="true" bounds="[517,2077][562,2122]" />\r\n              </node>\r\n            </node>\r\n          </node>\r\n        </node>\r\n      </node>\r\n    </node>\r\n  </node>\r\n</hierarchy>"""

In [ ]:
hypn_nodes = 200
hypn_steps = 2
agent_obj = NeuralAgent(spec_dims=19, maxn_nodes=hypn_nodes).to(device)
optimizer_obj = torch.optim.SGD(agent_obj.parameters(), lr=0.1)
for ep in range(100):
    print("# ep{}".format(ep))
    rollout(apk_obj, agent_obj, optimizer_obj, hypn_steps, hypn_nodes, debug_final_state)